In [1]:
import os
import numpy as np
import pandas as pd
# from models.load import TrainedModels

from utils.eval import save_iou_results
from utils.engine import get_iou_types, evaluate
from models.load import get_trained_model
from utils.eval import get_ap_ar, get_num_fps, get_num_fns, get_num_tps
from utils.print import print_title
from utils.init import reproducibility, clean_memory_get_device
from data.load import get_datasets, get_dataloaders
from data.paths import MIMIC_EYE_PATH
from tqdm import tqdm
from utils.train import  get_coco_eval_params
from utils.coco_eval import get_eval_params_dict

## Suppress the assignement warning from pandas.
pd.options.mode.chained_assignment = None  # default='warn

## Supress user warning
import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

In [2]:
def get_ap_ar_str(eval_dict, p):
    ap = eval_dict['ap']
    iouThr = eval_dict['iouThr']
    areaRng = eval_dict['areaRng']
    maxDets = eval_dict['maxDets']
    mean_s = eval_dict['mean_s']

    iStr = ' {:<18} {} @[ IoU={:<9} | area={:>6s} | maxDets={:>3d} ] = {:0.3f}'
    titleStr = 'Average Precision' if ap == 1 else 'Average Recall'
    typeStr = '(AP)' if ap==1 else '(AR)'
    iouStr = '{:0.2f}:{:0.2f}'.format(p.iouThrs[0], p.iouThrs[-1]) \
        if iouThr is None else '{:0.2f}'.format(iouThr)
    return iStr.format(titleStr, typeStr, iouStr, areaRng, maxDets, mean_s)


In [3]:
from enum import Enum

class TrainedModels(Enum):
    # mobilenet_baseline = "val_lesion-detection_ap_0_1655_test_lesion-detection_ap_0_1648_epoch50_03-15-2023 16-43-54_lesion_dsetection_baseline_mobilenet"  # mobilenet baseline
    # mobilenet_with_fix = "val_lesion-detection_ap_0_1918_test_lesion-detection_ap_0_1903_epoch16_03-16-2023 11-34-10_lesion_dsetection_with_fixation_mobilenet"
    # resnet18_baseline = "val_lesion-detection_ap_0_1973_test_lesion-detection_ap_0_2010_epoch22_03-16-2023 19-44-55_lesion_dsetection_baseline_resnet"
    # resnet18_with_fix = "val_lesion-detection_ap_0_1951_test_lesion-detection_ap_0_2195_epoch12_03-17-2023 00-31-54_lesion_dsetection_with_fixation_resnet"
    # densenet161_baseline = "val_lesion-detection_ap_0_1990_test_lesion-detection_ap_0_2085_epoch5_03-17-2023 08-53-33_lesion_dsetection_baseline_densenet161"
    # densenet161_with_fix = "val_lesion-detection_ap_0_2120_test_lesion-detection_ap_0_2104_epoch12_03-17-2023 18-36-01_lesion_dsetection_with_fixation_densenet161"
    # efficientnet_b5_baseline = "val_lesion-detection_ap_0_1898_test_lesion-detection_ap_0_2055_epoch5_03-17-2023 23-30-57_lesion_dsetection_baseline_efficientnet_b5"
    # efficientnet_b5_with_fix = "val_lesion-detection_ap_0_2117_test_lesion-detection_ap_0_2190_epoch8_03-18-2023 12-29-20_lesion_dsetection_with_fixation_efficientnet_b5"
    # efficientnet_b0_baseline = "val_lesion-detection_ap_0_1934_test_lesion-detection_ap_0_1858_epoch10_03-18-2023 23-50-47_lesion_dsetection_baseline_efficientnet_b0"
    # efficientnet_b0_with_fix = "val_lesion-detection_ap_0_2191_test_lesion-detection_ap_0_2162_epoch10_03-18-2023 19-38-11_lesion_dsetection_with_fixation_efficientnet_b0"
    convnext_base_with_fix = "val_lesion-detection_ap_0_2610_test_lesion-detection_ap_0_2548_epoch22_03-22-2023 02-55-37_lesion_dsetection_with_fixation_convnext_base"
    convnext_base_baseline = "val_lesion-detection_ap_0_2426_test_lesion-detection_ap_0_2325_epoch20_03-22-2023 11-53-53_lesion_dsetection_baseline_convnext_base"
    # vgg16_with_fix = "val_lesion-detection_ap_0_2301_test_lesion-detection_ap_0_2186_epoch22_03-20-2023 19-26-02_lesion_dsetection_with_fixation_vgg16"
    # vgg16_baseline = "val_lesion-detection_ap_0_2113_test_lesion-detection_ap_0_2068_epoch12_03-21-2023 00-45-24_lesion_dsetection_baseline_vgg16"
    # regnet_y_8gf_with_fix = "val_lesion-detection_ap_0_2267_test_lesion-detection_ap_0_2029_epoch12_03-21-2023 11-28-48_lesion_dsetection_with_fixation_regnet_y_8gf"
    # regnet_y_8gf_baseline = "val_lesion-detection_ap_0_1883_test_lesion-detection_ap_0_1658_epoch13_03-21-2023 15-22-32_lesion_dsetection_baseline_regnet_y_8gf"

In [4]:
normal_iou_thrs = np.array([0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95])
score_thresholds = [0.05]

In [5]:
from data.strs import SourceStrs, TaskStrs

# just generate for the test set.

for select_model in tqdm(list(TrainedModels)):

    for score_thrs in score_thresholds:

        device = clean_memory_get_device()
        reproducibility()

        model, train_info, _, _ = get_trained_model(
            select_model,
            device,
        )
        model = model.to(device)
        model.eval()

        setup = train_info.model_setup
        iou_types = get_iou_types(model, setup)

        dataset_params_dict = {
            "MIMIC_EYE_PATH": MIMIC_EYE_PATH,
            "labels_cols": setup.lesion_label_cols,
            "with_xrays_input": SourceStrs.XRAYS in setup.sources,
            "with_clincal_input": SourceStrs.CLINICAL in setup.sources,
            "with_fixations_input": SourceStrs.FIXATIONS in setup.sources,
            "fixations_mode_input": setup.fiaxtions_mode_input,
            "with_bboxes_label": TaskStrs.LESION_DETECTION in setup.tasks,
            "with_fixations_label": TaskStrs.FIXATION_GENERATION in setup.tasks,
            "fixations_mode_label": setup.fiaxtions_mode_label,
            "with_chexpert_label": TaskStrs.CHEXPERT_CLASSIFICATION in setup.tasks,
            "with_negbio_label": TaskStrs.NEGBIO_CLASSIFICATION in setup.tasks,
            "clinical_numerical_cols": setup.clinical_num,
            "clinical_categorical_cols": setup.clinical_cat,
            "image_size": setup.image_size,
            "image_mean": setup.image_mean,
            "image_std": setup.image_std,
        }

        detect_eval_dataset, train_dataset, val_dataset, test_dataset = get_datasets(
            dataset_params_dict=dataset_params_dict,
        )

        train_dataloader, val_dataloader, test_dataloader = get_dataloaders(
            train_dataset,
            val_dataset,
            test_dataset,
            batch_size=setup.batch_size,
        )

        train_coco = None
        train_coco, val_coco, test_coco, _ = get_coco_eval_params(
            source_name=SourceStrs.XRAYS,
            task_name=TaskStrs.LESION_DETECTION,
            train_dataloader=train_dataloader,
            val_dataloader=val_dataloader,
            test_dataloader=test_dataloader,
            detect_eval_dataset=detect_eval_dataset,
            iou_thrs=np.array([0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95]),
            use_iobb=setup.use_iobb,
            maxDets=[1, 5, 10, 30, 100],
            # train_dataloader,
            # val_dataloader,
            # test_dataloader,
            # detect_eval_dataset,
            # setup.iou_thrs,
            # setup.use_iobb,
        )

        normal_range_eval_params_dict = get_eval_params_dict(
            detect_eval_dataset,
            iou_thrs=normal_iou_thrs,
        )

        all_cat_ids = [None] + [
            detect_eval_dataset.disease_to_idx(d)
            for d in detect_eval_dataset.labels_cols
        ]

        for cat_id in all_cat_ids:
            cat_ids = (
                [
                    detect_eval_dataset.disease_to_idx(d)
                    for d in detect_eval_dataset.labels_cols
                ]
                if cat_id is None
                else [cat_id]
            )

            if not (cat_ids is None):
                normal_range_eval_params_dict["bbox"].catIds = cat_ids
                # normal_range_eval_params_dict["segm"].catIds = cat_ids

            test_evaluator, _ = evaluate(
                setup=setup,
                model=model,
                data_loader=test_dataloader,
                device=device,
                params_dict=normal_range_eval_params_dict,
                coco=test_coco,
                iou_types=iou_types,
                # score_thres=score_thres,
            )

            if cat_id is None:
                disease_str = "all"
            else:
                disease_str = detect_eval_dataset.label_idx_to_disease(cat_id)

            save_iou_results(
                test_evaluator['lesion-detection'],
                f"test_{disease_str}_score_thrs{score_thrs}",
                select_model.value,
            )

            test_ap_ar = get_ap_ar(
                test_evaluator['lesion-detection'],
                areaRng="all",
                maxDets=test_evaluator['lesion-detection'].coco_eval['bbox'].params.maxDets[-1],
                iouThr=None,
            )

            # test_ap_ar = get_ap_ar(
            #     test_evaluator, areaRng="all", maxDets=10, iouThr=0.5,
            # )
            # val_ap_ar = get_ap_ar(val_evaluator, areaRng="all", maxDets=10, iouThr=0.5,)

            # df = pd.DataFrame(
            #     [
            #         {
            #             f"AP@[.50:.05:.95]": test_ap_ar["ap"],
            #             f"AR@[.50:.05:.95]": test_ap_ar["ar"],
            #         },
            #     ]
            # )

            # df.to_csv(
            #     os.path.join(
            #         "eval_results",
            #         f"{select_model.value}_{disease_str}_score_thrs{score_thrs}.csv",
            #     )
            # )

            # num_fps = get_num_fps(
            #     test_evaluator["lesion-detection"],
            #     areaRng="all",
            #     maxDets=10,
            #     iouThr=0.5,
            # )

            # num_fns = get_num_fns(
            #     test_evaluator["lesion-detection"],
            #     areaRng="all",
            #     maxDets=10,
            #     iouThr=0.5,
            # )

            # num_tps = get_num_tps(
            #     test_evaluator["lesion-detection"],
            #     areaRng="all",
            #     maxDets=10,
            #     iouThr=0.5,
            # )

            coco_eval_str = ""
            for state in test_evaluator['lesion-detection'].coco_eval['bbox'].stats:
                coco_eval_str += f"{get_ap_ar_str(state,test_evaluator['lesion-detection'].coco_eval['bbox'].params)}\n"

            df = pd.DataFrame(
                [
                    {
                        "num_fps": test_evaluator['lesion-detection'].coco_eval['bbox'].eval['num_fps'], # @all_range, maxDet= p.maxDets[-1]
                        "num_fns": test_evaluator['lesion-detection'].coco_eval['bbox'].eval['num_fns'],
                        "num_tps": test_evaluator['lesion-detection'].coco_eval['bbox'].eval['num_tps'],
                        f"AP@[.50:.05:.95]": test_ap_ar["ap"],
                        f"AR@[.50:.05:.95]": test_ap_ar["ar"],
                        "coco_eval_str": coco_eval_str,
                    },
                ]
            )

            df.to_csv(
                os.path.join(
                    "eval_results",
                    f"{select_model.value}_{disease_str}_score_thrs{score_thrs}.csv",
                )
            )

            print_title(disease_str)
            print(df)
            # print(confusion_df)


  0%|          | 0/2 [00:00<?, ?it/s]

This notebook will running on device: [CUDA]
Using pretrained backbone. convnext_base
Using pretrained backbone. convnext_base
Using SGD as optimizer with lr=0.01
creating index...
index created!
Evaluation:  [ 0/57]  eta: 0:03:35  loss: 0.3125 (0.3125)  lesion-detection_performer-object_detection_loss_classifier: 0.1690 (0.1690)  lesion-detection_performer-object_detection_loss_box_reg: 0.1000 (0.1000)  lesion-detection_performer-object_detection_loss_objectness: 0.0401 (0.0401)  lesion-detection_performer-object_detection_loss_rpn_box_reg: 0.0034 (0.0034)  model_time: 1679979008.0000 (1679978991.8292)  evaluator_time: 0.0000 (0.0000)  time: 3.7870  data: 1.9291  max mem: 2756
Evaluation:  [56/57]  eta: 0:00:02  loss: 0.1860 (0.1899)  lesion-detection_performer-object_detection_loss_classifier: 0.0989 (0.0987)  lesion-detection_performer-object_detection_loss_box_reg: 0.0696 (0.0691)  lesion-detection_performer-object_detection_loss_objectness: 0.0167 (0.0186)  lesion-detection_perfor

 50%|█████     | 1/2 [20:45<20:45, 1245.40s/it]

   num_fps  num_fns  num_tps  AP@[.50:.05:.95]  AR@[.50:.05:.95]  \
0      417      262      142          0.068304          0.219802   

                                       coco_eval_str  
0   Average Precision  (AP) @[ IoU=0.50:0.95 | ar...  
This notebook will running on device: [CUDA]
Using pretrained backbone. convnext_base
Using SGD as optimizer with lr=0.01
creating index...
index created!
Evaluation:  [ 0/57]  eta: 0:01:09  loss: 0.3014 (0.3014)  lesion-detection_performer-object_detection_loss_classifier: 0.1531 (0.1531)  lesion-detection_performer-object_detection_loss_box_reg: 0.1114 (0.1114)  lesion-detection_performer-object_detection_loss_objectness: 0.0335 (0.0335)  lesion-detection_performer-object_detection_loss_rpn_box_reg: 0.0033 (0.0033)  model_time: 1679979904.0000 (1679979961.7152)  evaluator_time: 0.0000 (0.0000)  time: 1.2106  data: 1.0010  max mem: 2756
Evaluation:  [56/57]  eta: 0:00:01  loss: 0.1989 (0.1976)  lesion-detection_performer-object_detection_loss

100%|██████████| 2/2 [31:05<00:00, 932.51s/it] 

   num_fps  num_fns  num_tps  AP@[.50:.05:.95]  AR@[.50:.05:.95]  \
0      359      242      162          0.073267          0.241089   

                                       coco_eval_str  
0   Average Precision  (AP) @[ IoU=0.50:0.95 | ar...  
